In [28]:
import pandas as pd 
import os
from dotenv import load_dotenv
import berserk
pd.get_option("display.max_colwidth", 200)
# load content of env
load_dotenv()

# load the token
lichess_token = os.getenv("my_token")

# using token 
session = berserk.TokenSession(lichess_token)
client = berserk.Client(session=session)

# get profile data from my personal account
mydata = client.users.get_public_data("Trequartistaa")
mydata_df = pd.json_normalize(mydata).transpose()
#print(mydata_df)

# history 
myhistoy = client.users.get_rating_history("Trequartistaa")
myhistoy_df = pd.json_normalize(myhistoy)
#print(myhistoy_df)

# top 10 
top10_players = client.users.get_all_top_10()
pd.json_normalize(top10_players)

,bullet,blitz,rapid,classical,ultraBullet,crazyhouse,chess960,kingOfTheHill,threeCheck,antichess,atomic,horde,racingKings
0,"[{'id': 'rebeccaharris', 'username': 'RebeccaH...","[{'id': 'cutemouse83', 'username': 'cutemouse8...","[{'id': 'drvitman', 'username': 'DrVitman', 'p...","[{'id': 'plemsovhoz', 'username': 'PlemSovhoz'...","[{'id': 'penguingim1', 'username': 'penguingim...","[{'id': 'grxbullet', 'username': 'GRXbullet', ...","[{'id': 'zhigalko_sergei', 'username': 'Zhigal...","[{'id': 'zhigalko_sergei', 'username': 'Zhigal...","[{'id': 'zhigalko_sergei', 'username': 'Zhigal...","[{'id': 'lmaoooooooooooooooo', 'username': 'Lm...","[{'id': 'chrisrapid', 'username': 'chrisrapid'...","[{'id': 'rayholt', 'username': 'RayHolt', 'per...","[{'id': 'bijiy', 'username': 'bijiy', 'perfs':..."


In [22]:
# top bullet 
top10_bullet = pd.json_normalize(top10_players, record_path="bullet") 
top10_bullet

,id,username,title,patron,perfs.bullet.rating,perfs.bullet.progress,online
0,rebeccaharris,RebeccaHarris,GM,True,3225,-11,NaN
1,muisback,muisback,GM,True,3208,-25,NaN
2,watneg,Watneg,GM,NaN,3176,16,NaN
3,penguingim1,penguingim1,GM,True,3174,9,NaN
4,night-king96,Night-King96,GM,True,3165,-16,NaN
5,mirakel05,Mirakel05,FM,NaN,3159,24,NaN
6,ediz_gurel,Ediz_Gurel,GM,NaN,3159,-11,NaN
7,heisenberg01,Heisenberg01,FM,NaN,3157,-22,NaN
8,aaryan_varshney,aaryan_varshney,FM,NaN,3153,3,True
9,aqua_blazing,Aqua_Blazing,NaN,NaN,3127,11,True


In [23]:
# top 10 blitz
top10_blitz = pd.json_normalize(top10_players, record_path="blitz") 
top10_blitz

,id,username,title,perfs.blitz.rating,perfs.blitz.progress,online
0,cutemouse83,cutemouse83,GM,2987,21,NaN
1,aspiringstar,aspiringstar,GM,2977,6,NaN
2,dr-cro,Dr-CRO,GM,2951,40,NaN
3,athena-pallada,athena-pallada,GM,2951,-15,NaN
4,ominous_octopus,ominous_octopus,NaN,2948,-27,NaN
5,aqua_blazing,Aqua_Blazing,NaN,2938,14,True
6,temp006,temp006,GM,2935,2,NaN
7,idabble,Idabble,NaN,2927,-7,NaN
8,kivurawnuru,Kivurawnuru,NaN,2922,20,NaN
9,howardxue,HowardXue,NaN,2922,10,NaN


In [24]:
# top 10 rapid
top10_rapid = pd.json_normalize(top10_players, record_path="rapid") 
top10_rapid

,id,username,title,online,perfs.rapid.rating,perfs.rapid.progress,patron
0,drvitman,DrVitman,GM,True,2947,17,NaN
1,jumpingknight20,jumpingknight20,NaN,NaN,2822,-25,NaN
2,tuzakli_egitim,Tuzakli_Egitim,FM,NaN,2815,0,NaN
3,afanoffun,afanoffun,CM,NaN,2803,8,NaN
4,kurald_galain,Kurald_Galain,NaN,True,2790,13,NaN
5,homabiorkro,HomabiorKro,NaN,NaN,2783,-25,NaN
6,savvavetokhin2009,SavvaVetokhin2009,IM,True,2778,47,True
7,roman_yanchenko,Roman_Yanchenko,FM,True,2764,61,NaN
8,chess_ark,Chess_Ark,NaN,NaN,2760,16,NaN
9,kingslayer_999,KINGSLAYER_999,NaN,True,2758,17,NaN


In [29]:
# top 10 classic 
top10_classic = pd.json_normalize(top10_players, record_path="classical")
top10_classic

,id,username,title,perfs.classical.rating,perfs.classical.progress,patron,online
0,plemsovhoz,PlemSovhoz,IM,2618,12,NaN,NaN
1,sharkfang,Sharkfang,NaN,2617,18,True,True
2,thepub,ThePuB,NaN,2492,4,NaN,NaN
3,kanake04,Kanake04,NaN,2471,25,NaN,NaN
4,konnov_oleg,Konnov_Oleg,NaN,2456,13,NaN,True
5,ojaijoao,OjaiJoao,FM,2450,1,NaN,NaN
6,vlad_lazarev79,Vlad_Lazarev79,NaN,2439,-19,NaN,NaN
7,kc6,kc6,IM,2428,31,NaN,True
8,likeyacutg,likeyacutg,NaN,2415,15,True,True
9,alp_arslan92,alp_arslan92,NaN,2414,-17,True,NaN


In [30]:
top10_ultrabullet = pd.json_normalize(top10_players, record_path="ultraBullet")
top10_ultrabullet 

,id,username,title,patron,perfs.ultraBullet.rating,perfs.ultraBullet.progress,online
0,penguingim1,penguingim1,GM,True,2633,12,NaN
1,iamstraw,iamstraw,GM,NaN,2595,18,NaN
2,dmitriychesser,Dmitriychesser,NaN,NaN,2541,14,NaN
3,ohanyaneminchess,OhanyanEminChess,GM,NaN,2511,-9,True
4,bardyk01,Bardyk01,FM,NaN,2504,-24,NaN
5,blazinq,Blazinq,FM,NaN,2500,28,NaN
6,hyo_lee,hyo_lee,NaN,NaN,2446,1,NaN
7,tamojerry,tamojerry,FM,NaN,2436,-1,NaN
8,pantoef,Pantoef,NaN,NaN,2432,19,NaN
9,afanoffun,afanoffun,CM,NaN,2414,-9,NaN
